In [1]:
# Packages
import requests
import pandas as pd
import time
import sqlite3
import json

# OpenSea

In [ ]:
def fetch_page(url, headers, params, max_retries=5):
    """
    Makes a GET request to the given URL with built-in retry logic for 5xx errors.
    Returns the JSON response or None if the retries fail.
    """
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, params=params)
            
            # If successful response, return the parsed JSON
            if response.status_code == 200:
                return response.json()
            
            # If it's a server error (5xx), we can retry
            elif 500 <= response.status_code < 600:
                print(f"Server error {response.status_code} on attempt {attempt+1}. Retrying...")
                time.sleep(2 ** attempt)  # Exponential backoff: 2^0=1s, 2^1=2s, 2^2=4s ...
            else:
                # For other error codes (400-range, etc.), decide what you want to do
                print(f"Unexpected error {response.status_code}: {response.text}")
                return None
        
        except requests.exceptions.RequestException as e:
            # Network errors, timeouts, etc.
            print(f"Request exception: {e}. Attempt {attempt+1}. Retrying...")
            time.sleep(2 ** attempt)
    
    # If we got here, all retries failed
    print(f"Failed after {max_retries} attempts. Returning None.")
    return None


def create_table_if_not_exists(conn):
    """
    Create a table named 'collections' if it doesn't exist.
    """
    
    create_sql = """
    CREATE TABLE IF NOT EXISTS collections (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        collection_slug TEXT,
        name TEXT,
        description TEXT,
        image_url TEXT,
        banner_image_url TEXT,
        owner TEXT,
        safelist_status TEXT,
        category TEXT,
        is_disabled INTEGER,
        is_nsfw INTEGER,
        trait_offers_enabled INTEGER,
        collection_offers_enabled INTEGER,
        opensea_url TEXT,
        project_url TEXT,
        wiki_url TEXT,
        discord_url TEXT,
        telegram_url TEXT,
        twitter_username TEXT,
        instagram_username TEXT,
        contract_info TEXT
    )
    """
    conn.execute(create_sql)
    conn.commit()


def insert_collections(conn, collections_data):
    """
    Insert a list of collection dicts into the 'collections' table.
    For the 'contracts' array, we only store the first item as 'address:chain'.
    """
    insert_sql = """
    INSERT INTO collections (
        collection_slug,
        name,
        description,
        image_url,
        banner_image_url,
        owner,
        safelist_status,
        category,
        is_disabled,
        is_nsfw,
        trait_offers_enabled,
        collection_offers_enabled,
        opensea_url,
        project_url,
        wiki_url,
        discord_url,
        telegram_url,
        twitter_username,
        instagram_username,
        contract_info
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    data_to_insert = []
    for item in collections_data:
        # Extract first contract address:chain if it exists
        contracts = item.get("contracts", [])
        if contracts:
            first_contract = contracts[0]
            contract_info = f"{first_contract.get('address','')}:{first_contract.get('chain','')}"
        else:
            contract_info = ""

        row = (
            item.get("collection", ""),  # collection_slug
            item.get("name", ""),
            item.get("description", ""),
            item.get("image_url", ""),
            item.get("banner_image_url", ""),
            item.get("owner", ""),
            item.get("safelist_status", ""),
            item.get("category", ""),
            1 if item.get("is_disabled", False) else 0,
            1 if item.get("is_nsfw", False) else 0,
            1 if item.get("trait_offers_enabled", False) else 0,
            1 if item.get("collection_offers_enabled", False) else 0,
            item.get("opensea_url", ""),
            item.get("project_url", ""),
            item.get("wiki_url", ""),
            item.get("discord_url", ""),
            item.get("telegram_url", ""),
            item.get("twitter_username", ""),
            item.get("instagram_username", ""),
            contract_info
        )
        data_to_insert.append(row)

    conn.executemany(insert_sql, data_to_insert)
    conn.commit()


def fetch_all_collections(api_key, max_pages=None):
    """
    Fetch as many NFT collections as possible from OpenSea using the 
    paginated /api/v2/collections endpoint. Each page gets written to
    opensea_collections.db immediately.
    
    :param api_key:    Your OpenSea API key
    :param max_pages:  Integer maximum pages to fetch, or None to keep going.
    """
    base_url = "https://api.opensea.io/api/v2/collections"
    headers = {
        "Accept": "application/json",
        "x-api-key": api_key
    }

    # Connect to SQLite DB (on disk). The file is created if it doesn't exist
    conn = sqlite3.connect("data/opensea_collections.db")
    create_table_if_not_exists(conn)

    page_count = 0
    next_cursor = None

    while True:
        params = {
            "limit": 100,
            "order_by": "created_date"
        }
        if next_cursor:
            params["next"] = next_cursor

        data = fetch_page(base_url, headers, params, max_retries=5)
        if not data:
            print("No data returned (or retries exhausted). Stopping.")
            break
        
        collections_data = data.get("collections", [])
        if not collections_data:
            print("No more collections returned. Stopping.")
            break

        # Write the fetched collections into the DB
        insert_collections(conn, collections_data)

        page_count += 1
        print(f"Fetched {len(collections_data)} collections on page {page_count}.")

        # If we have a max_pages limit, stop once reached
        if max_pages and page_count >= max_pages:
            print(f"Reached max_pages={max_pages}. Stopping.")
            break

        # Check for the 'next' cursor for pagination
        next_cursor = data.get("next")
        if not next_cursor:
            print("No next cursor returned. Completed pagination.")
            break

        # Optional small delay to avoid rate limiting
        # time.sleep(0.25)

    conn.close()
    print(f"Finished. Pages fetched: {page_count}")

In [ ]:
# Execution

if __name__ == "__main__":
    OPENSEA_API_KEY = "your_key"
    fetch_all_collections(OPENSEA_API_KEY, max_pages=30000)

Fetched 100 collections on page 1.
Fetched 100 collections on page 2.
Fetched 100 collections on page 3.
Fetched 100 collections on page 4.
Fetched 100 collections on page 5.
Fetched 100 collections on page 6.
Fetched 100 collections on page 7.
Fetched 100 collections on page 8.
Fetched 100 collections on page 9.
Fetched 100 collections on page 10.
Fetched 100 collections on page 11.
Fetched 100 collections on page 12.
Fetched 100 collections on page 13.
Fetched 100 collections on page 14.
Fetched 100 collections on page 15.
Fetched 100 collections on page 16.
Fetched 100 collections on page 17.
Fetched 100 collections on page 18.
Fetched 100 collections on page 19.
Fetched 100 collections on page 20.
Fetched 100 collections on page 21.
Fetched 100 collections on page 22.
Fetched 100 collections on page 23.
Fetched 100 collections on page 24.
Fetched 100 collections on page 25.
Fetched 100 collections on page 26.
Fetched 100 collections on page 27.
Fetched 100 collections on page 28.
F

Supply and Fees

In [ ]:
def safe_get(url, headers, max_retries=5):
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, timeout=15)
            
            if response.status_code == 200:
                return response.json()
            elif 500 <= response.status_code < 600:
                print(f"Server error {response.status_code}, attempt {attempt+1}. Retrying...")
                time.sleep(2 ** attempt)  # exponential backoff
            else:
                # 4xx or other error => skip
                print(f"Error {response.status_code} fetching {url}: {response.text}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"RequestException on attempt {attempt+1}: {e}")
            time.sleep(2 ** attempt)

    print(f"Failed after {max_retries} attempts: {url}")
    return None

def add_missing_columns(conn):
    """
    Adds the new columns to the existing 'collections' table.
    If the columns already exist, SQLite will ignore the error
    (or you can wrap in try/except).
    """
    cursor = conn.cursor()
    
    # For each column, attempt an ALTER TABLE statement
    # (If the column exists, this will error, so you may want try/except)
    alter_statements = [
        "ALTER TABLE collections ADD COLUMN total_supply INTEGER",
        "ALTER TABLE collections ADD COLUMN created_date TEXT",
        "ALTER TABLE collections ADD COLUMN opensea_fee REAL",
        "ALTER TABLE collections ADD COLUMN opensea_fee_mandatory INTEGER",
        "ALTER TABLE collections ADD COLUMN royalty REAL",
        "ALTER TABLE collections ADD COLUMN royalty_mandatory INTEGER"
    ]
    
    for stmt in alter_statements:
        try:
            cursor.execute(stmt)
            print(f"Executed: {stmt}")
        except sqlite3.OperationalError as e:
            # Likely the column already exists
            print(f"Skipping: {stmt} => {e}")
    
    conn.commit()

def update_opensea_details(conn, api_key):
    """
    Reads 'id' and 'collection_slug' from 'collections' table
    in 'collections_opensea.db', calls
    https://api.opensea.io/api/v2/collections/{collection_slug}
    then updates the record with the new fields:
      - total_supply
      - created_date
      - opensea_fee
      - opensea_fee_mandatory
      - royalty
      - royalty_mandatory
    """
    cursor = conn.cursor()
    
    # fetch all slugs
    cursor.execute("SELECT id, collection_slug FROM collections")
    rows = cursor.fetchall()
    
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    
    update_sql = """
    UPDATE collections
    SET total_supply = ?,
        created_date = ?,
        opensea_fee = ?,
        opensea_fee_mandatory = ?,
        royalty = ?,
        royalty_mandatory = ?
    WHERE id = ?
    """
    
    count = 0
    for (row_id, slug) in rows:
        if not slug:
            # If there's no slug, skip
            continue
        
        url = f"https://api.opensea.io/api/v2/collections/{slug}"
        
        data = safe_get(url, headers)
        if not data:
            # Could not retrieve or error
            continue
        
        # parse the fields
        # top-level object is the 'collection' key from the JSON
        collection_obj = data  # might be 'data["collection"]' but the sample shows root-level keys
        # Check if 'collection' is the actual root or there's a nested?
        # If the API returns: { "collection": {...} }, we do:
        if "collection" in data and isinstance(data["collection"], dict):
            collection_obj = data["collection"]
        
        # total_supply
        total_supply = collection_obj.get("total_supply", 0)
        
        # created_date
        created_date = collection_obj.get("created_date", "")
        
        # fees array
        fees = collection_obj.get("fees", [])
        
        opensea_fee = 0.0
        opensea_fee_mandatory = 0
        royalty = 0.0
        royalty_mandatory = 0
        
        if len(fees) >= 1:
            opensea_fee = float(fees[0].get("fee", 0.0))
            opensea_fee_mandatory = 1 if fees[0].get("required", False) else 0
        
        if len(fees) >= 2:
            royalty = float(fees[1].get("fee", 0.0))
            royalty_mandatory = 1 if fees[1].get("required", False) else 0
        
        # Now update the row
        cursor.execute(update_sql, (
            total_supply,
            created_date,
            opensea_fee,
            opensea_fee_mandatory,
            royalty,
            royalty_mandatory,
            row_id
        ))
        count += 1
        
        # optional: slow down to avoid rate-limiting
        # time.sleep(0.1)
    
    conn.commit()
    print(f"Updated {count} rows with new data.")


def main():
    api_key = "your_key"
    db_path = "data/collections_opensea.db"
    
    conn = sqlite3.connect(db_path)
    
    # Add columns if needed
    add_missing_columns(conn)
    
    # Update each row with extended info
    update_opensea_details(conn, api_key)
    
    conn.close()
    print("Done.")

In [3]:
if __name__ == "__main__":
    main()

Skipping: ALTER TABLE collections ADD COLUMN total_supply INTEGER => duplicate column name: total_supply
Skipping: ALTER TABLE collections ADD COLUMN created_date TEXT => duplicate column name: created_date
Skipping: ALTER TABLE collections ADD COLUMN opensea_fee REAL => duplicate column name: opensea_fee
Skipping: ALTER TABLE collections ADD COLUMN opensea_fee_mandatory INTEGER => duplicate column name: opensea_fee_mandatory
Skipping: ALTER TABLE collections ADD COLUMN royalty REAL => duplicate column name: royalty
Skipping: ALTER TABLE collections ADD COLUMN royalty_mandatory INTEGER => duplicate column name: royalty_mandatory
Error 400 fetching https://api.opensea.io/api/v2/collections/world-coins-2: {"errors":["Collection world-coins-2 not found"]}
Error 400 fetching https://api.opensea.io/api/v2/collections/lord-of-toil-night-1: {"errors":["Collection lord-of-toil-night-1 not found"]}
Error 400 fetching https://api.opensea.io/api/v2/collections/cl-01: {"errors":["Collection cl-01 

KeyboardInterrupt: 

# Rarible

In [ ]:
import requests
import time
import sqlite3

def create_table_if_not_exists(conn):
    """
    Create a 'rarible_collections' table if it does not exist.
    You can adjust the columns as you see fit.
    """
    create_sql = """
    CREATE TABLE IF NOT EXISTS rarible_collections (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        collection_id TEXT,
        blockchain TEXT,
        structure TEXT,
        collection_type TEXT,
        status TEXT,
        name TEXT,
        symbol TEXT,
        owner TEXT,
        meta_name TEXT,
        meta_description TEXT,
        meta_created_at TEXT,
        spam_score INTEGER,
        is_shared INTEGER,
        is_self INTEGER,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    conn.execute(create_sql)
    conn.commit()


def insert_collections(conn, collections):
    """
    Insert a batch of collections into the 'rarible_collections' table.
    """
    insert_sql = """
    INSERT INTO rarible_collections (
        collection_id,
        blockchain,
        structure,
        collection_type,
        status,
        name,
        symbol,
        owner,
        meta_name,
        meta_description,
        meta_created_at,
        spam_score,
        is_shared,
        is_self
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    records = []
    for c in collections:
        collection_id = c.get("id", "")
        blockchain = c.get("blockchain", "")
        structure = c.get("structure", "")
        collection_type = c.get("type", "")
        status = c.get("status", "")
        name = c.get("name", "")
        symbol = c.get("symbol", "")
        owner = c.get("owner", "")
        
        meta = c.get("meta", {})
        meta_name = meta.get("name", "")
        meta_description = meta.get("description", "")
        meta_created_at = meta.get("createdAt", "")
        
        spam_score = c.get("spamScore", 0)
        is_shared = 1 if c.get("shared", False) else 0
        is_self_value = 1 if c.get("self", False) else 0
        
        row = (
            collection_id,
            blockchain,
            structure,
            collection_type,
            status,
            name,
            symbol,
            owner,
            meta_name,
            meta_description,
            meta_created_at,
            spam_score,
            is_shared,
            is_self_value
        )
        records.append(row)

    conn.executemany(insert_sql, records)
    conn.commit()


def fetch_all_rarible_collections(api_key, max_pages=None):
    """
    Fetch as many collections as possible from Rarible's /v0.1/collections/all endpoint
    using continuation-based pagination. Each page is saved immediately into a SQLite
    database ('collections_rarible.db') so that data isn't lost if the script is interrupted.

    :param api_key: Your Rarible API key (string).
    :param max_pages: Optional integer limit on pages to fetch. If None, continue until no continuation.
    """
    base_url = "https://api.rarible.org/v0.1/collections/all"

    headers = {
        "accept": "application/json",
        "X-API-KEY": api_key
    }

    # Connect to the local SQLite database
    conn = sqlite3.connect("data/collections_rarible.db")
    create_table_if_not_exists(conn)

    continuation = None
    page_count = 0

    while True:
        params = {
            "size": 100  # up to 100 items per page (you can adjust as needed)
        }
        if continuation:
            params["continuation"] = continuation

        try:
            response = requests.get(base_url, headers=headers, params=params)
        except requests.exceptions.RequestException as e:
            print(f"Request exception: {e}. Stopping.")
            break

        if response.status_code != 200:
            print(f"Error code {response.status_code}: {response.text}")
            break

        data = response.json()

        # Extract the collections from this page
        collections = data.get("collections", [])
        if not collections:
            print("No collections returned. Stopping.")
            break

        # Insert them into the DB
        insert_collections(conn, collections)

        page_count += 1
        print(f"Fetched {len(collections)} collections on page {page_count}.")

        # Check if we have a next page (continuation token)
        continuation = data.get("continuation")
        if not continuation:
            print("No more 'continuation' token, finished all pages.")
            break

        # If you have a maximum page limit
        if max_pages and page_count >= max_pages:
            print(f"Reached max_pages={max_pages}. Stopping.")
            break

        # OPTIONAL: small delay to avoid hitting rate limits or server overload
        # time.sleep(0.1)

    conn.close()
    print(f"Finished. Pages fetched: {page_count}")

In [ ]:
if __name__ == "__main__":
    # Replace with your real Rarible API key
    RARIBLE_API_KEY = "your_key"

    # Run the fetch function
    # For unlimited pages, set max_pages=None, or pick a limit like 100:
    fetch_all_rarible_collections(RARIBLE_API_KEY, max_pages=None)

Fetched 100 collections on page 1.
Fetched 100 collections on page 2.
Fetched 100 collections on page 3.
Fetched 100 collections on page 4.
Fetched 100 collections on page 5.
Fetched 100 collections on page 6.
Fetched 100 collections on page 7.
Fetched 100 collections on page 8.
Fetched 100 collections on page 9.
Fetched 100 collections on page 10.
Fetched 100 collections on page 11.
Fetched 100 collections on page 12.
Fetched 100 collections on page 13.
Fetched 100 collections on page 14.
Fetched 100 collections on page 15.
Fetched 100 collections on page 16.
Fetched 100 collections on page 17.
Fetched 100 collections on page 18.
Fetched 100 collections on page 19.
Fetched 100 collections on page 20.
Fetched 100 collections on page 21.
Fetched 100 collections on page 22.
Fetched 100 collections on page 23.
Fetched 100 collections on page 24.
Fetched 100 collections on page 25.
Fetched 100 collections on page 26.
Fetched 100 collections on page 27.
Fetched 100 collections on page 28.
F

# Magic Eden

In [ ]:

def create_table_if_not_exists(conn):
    create_sql = """
    CREATE TABLE IF NOT EXISTS magiceden_collections (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        symbol TEXT,
        name TEXT,
        description TEXT,
        image TEXT,
        twitter TEXT,
        discord TEXT,
        website TEXT,
        categories TEXT,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    conn.execute(create_sql)
    conn.commit()

def insert_collections(conn, collections_data):
    insert_sql = """
    INSERT INTO magiceden_collections (
        symbol,
        name,
        description,
        image,
        twitter,
        discord,
        website,
        categories
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """

    records = []
    for item in collections_data:
        symbol = item.get("symbol", "")
        name = item.get("name", "")
        description = item.get("description", "")
        image = item.get("image", "")
        twitter = item.get("twitter", "")
        discord = item.get("discord", "")
        website = item.get("website", "")
        # store categories as JSON string (or comma-separated, up to you)
        categories_str = json.dumps(item.get("categories", []))

        row = (symbol, name, description, image, twitter, discord, website, categories_str)
        records.append(row)

    conn.executemany(insert_sql, records)
    conn.commit()

def fetch_all_magiceden_collections():
    conn = sqlite3.connect("data/collections_magiceden.db")
    create_table_if_not_exists(conn)

    base_url = "https://api-mainnet.magiceden.dev/v2/collections"
    limit = 100
    offset = 0
    page_count = 0

    while True:
        params = {
            "offset": offset,
            "limit": limit
        }
        headers = {
            "accept": "application/json",
            "paging": "true"
        }

        try:
            response = requests.get(base_url, headers=headers, params=params)
        except requests.exceptions.RequestException as e:
            print(f"Request exception: {e}. Stopping.")
            break

        if response.status_code == 200:
            data = response.json()
            if not data:
                print("No more collections returned. Stopping.")
                break

            # Insert into DB
            insert_collections(conn, data)

            page_count += 1
            print(f"Fetched {len(data)} collections on page {page_count} (offset={offset}).")

            # If we got fewer than limit, presumably no more data
            if len(data) < limit:
                print("Received fewer than 'limit' items. Stopping pagination.")
                break

            offset += limit

        elif response.status_code == 400:
            # We got the "offset and limit must be multiples of 20" or
            # "offset must be a multiple of the limit" message
            print(f"400 Bad Request: {response.text}")
            print("Likely reached an internal offset limit or no more data is available.")
            break
        else:
            print(f"Non-200 status code {response.status_code}. Response: {response.text}")
            break

        # optional small delay
        time.sleep(0.1)

    conn.close()
    print(f"Finished scraping. Total pages: {page_count}")


In [3]:
if __name__ == "__main__":
    fetch_all_magiceden_collections()

Fetched 100 collections on page 1 (offset=0).
Fetched 100 collections on page 2 (offset=100).
Fetched 100 collections on page 3 (offset=200).
Fetched 100 collections on page 4 (offset=300).
Fetched 100 collections on page 5 (offset=400).
Fetched 100 collections on page 6 (offset=500).
Fetched 100 collections on page 7 (offset=600).
Fetched 100 collections on page 8 (offset=700).
Fetched 100 collections on page 9 (offset=800).
Fetched 100 collections on page 10 (offset=900).
Fetched 100 collections on page 11 (offset=1000).
Fetched 100 collections on page 12 (offset=1100).
Fetched 100 collections on page 13 (offset=1200).
Fetched 100 collections on page 14 (offset=1300).
Fetched 100 collections on page 15 (offset=1400).
Fetched 100 collections on page 16 (offset=1500).
Fetched 100 collections on page 17 (offset=1600).
Fetched 100 collections on page 18 (offset=1700).
Fetched 100 collections on page 19 (offset=1800).
Fetched 100 collections on page 20 (offset=1900).
Fetched 100 collectio

# Atomic Market

In [5]:
def create_table_if_not_exists(conn):
    """
    Creates a table 'atomic_collections' if it doesn't exist already.
    We'll store the fields returned by the API for each collection.
    """
    create_sql = """
    CREATE TABLE IF NOT EXISTS atomic_collections (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        contract TEXT,
        collection_name TEXT,
        name TEXT,
        author TEXT,
        allow_notify INTEGER,
        authorized_accounts TEXT,  -- will store as JSON array
        notify_accounts TEXT,      -- also JSON
        market_fee REAL,
        data_json TEXT,            -- entire data field as JSON
        created_at_block TEXT,
        created_at_time TEXT,
        inserted_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    conn.execute(create_sql)
    conn.commit()


def insert_collections(conn, collections):
    """
    Insert a list of collections into the 'atomic_collections' table.
    Each 'collection' is a dict with fields like:
        'contract', 'collection_name', 'name', 'author', 'allow_notify', etc.
    """
    insert_sql = """
    INSERT INTO atomic_collections (
        contract,
        collection_name,
        name,
        author,
        allow_notify,
        authorized_accounts,
        notify_accounts,
        market_fee,
        data_json,
        created_at_block,
        created_at_time
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    records = []
    for coll in collections:
        contract = coll.get("contract", "")
        collection_name = coll.get("collection_name", "")
        name = coll.get("name", "")
        author = coll.get("author", "")
        
        # Booleans can be stored as 0/1 in SQLite
        allow_notify = 1 if coll.get("allow_notify", False) else 0
        
        # authorized_accounts and notify_accounts are arrays
        authorized_accounts = json.dumps(coll.get("authorized_accounts", []))
        notify_accounts = json.dumps(coll.get("notify_accounts", []))
        
        market_fee = coll.get("market_fee", 0.0)
        
        # 'data' is an object with dynamic fields. Store as JSON
        data_field = coll.get("data", {})
        data_json = json.dumps(data_field)
        
        created_at_block = coll.get("created_at_block", "")
        created_at_time = coll.get("created_at_time", "")
        
        record = (
            contract,
            collection_name,
            name,
            author,
            allow_notify,
            authorized_accounts,
            notify_accounts,
            market_fee,
            data_json,
            created_at_block,
            created_at_time
        )
        records.append(record)
    
    conn.executemany(insert_sql, records)
    conn.commit()


def fetch_atomic_collections(page=1, limit=100, order="desc", sort="created"):
    """
    Fetch collections for a given page from the AtomicAssets /v1/collections endpoint.
    Returns the parsed JSON 'data' if success, or None if an error occurs.
    
    By default:
        page=1, limit=100, order=desc, sort=created
    """
    base_url = "https://wax.api.atomicassets.io/atomicassets/v1/collections"
    params = {
        "page": page,
        "limit": limit,
        "order": order,
        "sort": sort
    }
    try:
        response = requests.get(base_url, params=params, timeout=15)
        if response.status_code != 200:
            print(f"Non-200 status: {response.status_code} => {response.text}")
            return None
        
        data = response.json()
        # data structure: { "success": true/false, "data": [...], "query_time": 0 }
        if not data.get("success", False):
            print("API returned success=false.")
            return None
        
        return data.get("data", [])
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None


def scrape_atomic_collections(max_pages=None):
    """
    Paginated loop to fetch all possible collections (or up to max_pages if set).
    Each page has up to 100 results. We'll stop once we get an empty list or reach max_pages.
    """
    conn = sqlite3.connect("atomic_collections.db")
    create_table_if_not_exists(conn)

    page = 1
    total_fetched = 0

    while True:
        collections_data = fetch_atomic_collections(page=page, limit=100, order="desc", sort="created")
        if not collections_data:
            print(f"No more data returned or error on page {page}, stopping.")
            break
        
        # Insert into DB
        insert_collections(conn, collections_data)
        fetched_now = len(collections_data)
        total_fetched += fetched_now
        print(f"Page {page} fetched {fetched_now}, total so far: {total_fetched}")

        # If fewer than 100 returned, we might be at the end
        if fetched_now < 100:
            print("Fewer than 100 returned, presumably last page.")
            break
        
        page += 1

        # Optional: If you have a max_pages limit
        if max_pages and page > max_pages:
            print(f"Reached max_pages={max_pages}. Stopping.")
            break

        # Optional short delay to avoid rate-limiting or be kind to server
        # time.sleep(0.2)
    
    conn.close()
    print(f"Finished scraping. Total collections fetched: {total_fetched}")



In [6]:
if __name__ == "__main__":
    # Example: collect all pages until no more data or an error arises
    scrape_atomic_collections(max_pages=None)

Page 1 fetched 100, total so far: 100
Page 2 fetched 100, total so far: 200
Page 3 fetched 100, total so far: 300
Page 4 fetched 100, total so far: 400
Page 5 fetched 100, total so far: 500
Page 6 fetched 100, total so far: 600
Page 7 fetched 100, total so far: 700
Page 8 fetched 100, total so far: 800
Page 9 fetched 100, total so far: 900
Page 10 fetched 100, total so far: 1000
Page 11 fetched 100, total so far: 1100
Page 12 fetched 100, total so far: 1200
Page 13 fetched 100, total so far: 1300
Page 14 fetched 100, total so far: 1400
Page 15 fetched 100, total so far: 1500
Page 16 fetched 100, total so far: 1600
Page 17 fetched 100, total so far: 1700
Page 18 fetched 100, total so far: 1800
Page 19 fetched 100, total so far: 1900
Page 20 fetched 100, total so far: 2000
Page 21 fetched 100, total so far: 2100
Page 22 fetched 100, total so far: 2200
Page 23 fetched 100, total so far: 2300
Page 24 fetched 100, total so far: 2400
Page 25 fetched 100, total so far: 2500
Page 26 fetched 10